In [2]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import re

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [410]:
# Load orginal dataset

index_data = pd.read_excel('C:/Users/fanjz/Desktop/dataset/nmi/nmi_level.xlsx')
glance = pd.read_excel('C:/Users/fanjz/Desktop/dataset/nmi/glancetable.xlsx')
comments = pd.read_excel('C:/Users/fanjz/Desktop/dataset/nmi/comments.xlsx')
sector_ranking = pd.read_excel('C:/Users/fanjz/Desktop/dataset/nmi/sector_ranking.xlsx')

In [411]:
# Copy and paste latest month data url

url = input('Please Paste Latest NMI URL: ')

url_str = str(url)

response = requests.get(url_str)

### Extract Data from URL ###

#### 1. Get comments from respondents ####

In [412]:
# Extract What Respondents Say

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        main_content = section.find('div', class_='col-lg-10 col-lg-offset-1')

        if main_content:
            ul_element = main_content.find('ul', type='disc')
            if ul_element:
                list_items = ul_element.find_all('li')
                if list_items:
                    # Create a list to store the text of each list item
                    items_list = [item.text.strip() for item in list_items]

                    # Create a DataFrame from the list
                    respondent_comment = pd.DataFrame(items_list, columns=['WHAT RESPONDENTS ARE SAYING'])
        else:
            print("Main content not found within the section.")
    else:
        print("Section with class 'xxxx' not found.")

else:
    print("Failed to retrieve the page")

#### 2. Get Monthly Level Data ####

In [413]:
# Extract monthly index level

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        table_div = section.find('div', class_='table-responsive')

        if table_div:
        # Find the table within the div
            table = table_div.find('table')

            if table:
                # Read the table into a DataFrame
                data_table = pd.read_html(str(table))[0]
            else:
                print("Table not found within the div with class 'table-responsive'")
        else:
            print("Div with class 'table-responsive' not found.")
    else:
        print("Failed to retrieve the page")

C:\Users\fanjz\AppData\Local\Temp\ipykernel_77868\2319469250.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]


#### 3. Get Commodity Data ####

In [414]:
# Extract Commodity up/down/short supply

commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(14)')
commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(15)')
commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(16)')

# commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(16)')
# commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(17)')
# commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(18)')

In [415]:
commodity_tags = [commodity_up,commodity_down,commodity_short]
commoditiy_direction = []
for i, tag in enumerate(commodity_tags):
    if tag:
        direction = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        commoditiy_direction.append(direction)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
commodity_comments = pd.DataFrame(commoditiy_direction, columns=['content'])

In [416]:
substring_patterns = ['Commodities Up in Price','Commodities Down in Price','Commodities in']

def split_based_on_pattern(content, pattern):
    if pattern in content:
        parts = content.split(pattern, 1)
        return [pattern.strip(), parts[1].strip()]
    else:
        return [content.strip(), '']

# Split each row based on the defined patterns
split_data = []
for i, row in commodity_comments.iterrows():
    split_data.append(split_based_on_pattern(row['content'], substring_patterns[i]))

# Create a new DataFrame from the split data
commodity_df = pd.DataFrame(split_data, columns=['pattern', 'rest'])

# Display the split DataFrame
commodity_df

,pattern,rest
0,Commodities Up in Price,"Caustic Soda; Construction Contractors (4); Copper Wire (2); Diesel Fuel (2); Food and Beverages; Fuel (3); Gasoline (3); Heating, Ventilation and..."
1,Commodities Down in Price,Propane.
2,Commodities in,Short Supply Brass Fittings; Electrical Components (4); Electrical Equipment (2); Labor (3); Switchgear (2); and Transformers.


#### 4. Get Industry Ranking #### 

In [417]:
# # Extract Industry Rankings

nmi = soup.select_one('#main > article > section > div > div > p:nth-child(8)')
businesss_activity = soup.select_one('#main > article > section > div > div > p:nth-child(24)')
new_order = soup.select_one('#main > article > section > div > div > p:nth-child(27)')
employment= soup.select_one('#main > article > section > div > div > p:nth-child(30)')
supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(33)')
inventories = soup.select_one('#main > article > section > div > div > p:nth-child(36)')
prices = soup.select_one('#main > article > section > div > div > p:nth-child(39)')
backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(43)')
new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(46)')
imports = soup.select_one('#main > article > section > div > div > p:nth-child(49)')
inventory_sentiment = soup.select_one('#main > article > section > div > div > p:nth-child(52)')

# # Extract Industry Rankings

# nmi = soup.select_one('#main > article > section > div > div > p:nth-child(8)')
# businesss_activity = soup.select_one('#main > article > section > div > div > p:nth-child(25)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(28)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(31)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(34)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(37)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(40)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(44)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(47)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(50)')
# inventory_sentiment = soup.select_one('#main > article > section > div > div > p:nth-child(53)')

# # Extract Industry Rankings

# nmi = soup.select_one('#main > article > section > div > div > p:nth-child(10)')
# businesss_activity = soup.select_one('#main > article > section > div > div > p:nth-child(26)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(29)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(32)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(35)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(38)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(41)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(45)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(48)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(51)')
# inventory_sentiment = soup.select_one('#main > article > section > div > div > p:nth-child(54)')

# Extract Industry Rankings

# pmi = soup.select_one('#main > article > section > div > div > p:nth-child(11)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(27)')
# production = soup.select_one('#main > article > section > div > div > p:nth-child(30)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(33)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(36)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(39)')
# customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(42)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(45)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(48)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(51)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(54)')



In [418]:
industry_ranking = [nmi,businesss_activity,new_order,employment,supplier_deliveries,inventories,prices,backlog_orders,new_export_orders,imports, inventory_sentiment]

rankings = []
for i, tag in enumerate(industry_ranking):
    if tag:
        ranking = tag.get_text().strip()  # Get all the text within the <p> tag
        rankings.append(ranking)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
ranking_df = pd.DataFrame(rankings, columns=['Ranking'])
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip()


In [419]:
industry_list = ['NMI','Business Activity','New Order','Employment','Supplier Deliveries','Inventories','Prices','Backlog Orders','New Export Orders','Imports','Inventory Sentiment']
ranking_df['Industry'] = industry_list
col_oder = ['Industry','Ranking']
ranking_df = ranking_df[col_oder]

In [420]:
ranking_df

,Industry,Ranking
0,NMI,INDUSTRY PERFORMANCEThe 12 services industries reporting growth in April — listed in order — are: Accommodation & Food Services; Construction; Agr...
1,Business Activity,The 13 industries reporting an increase in business activity for the month of April — listed in order — are: Accommodation & Food Services; Agricu...
2,New Order,"The 12 industries reporting an increase in new orders for the month of April — listed in order — are: Accommodation & Food Services; Agriculture, ..."
3,Employment,The five industries reporting an increase in employment in April are: Accommodation & Food Services; Management of Companies & Support Services; C...
4,Supplier Deliveries,The five industries reporting slower deliveries in April are: Retail Trade; Educational Services; Health Care & Social Assistance; Construction; a...
5,Inventories,"The six industries reporting an increase in inventories in April — in the following order — are: Retail Trade; Agriculture, Forestry, Fishing & Hu..."
6,Prices,"Fourteen services industries reported an increase in prices paid during the month of April, in the following order: Construction; Management of Co..."
7,Backlog Orders,"The nine industries reporting an increase in order backlogs in April, in order, are: Agriculture, Forestry, Fishing & Hunting; Real Estate, Rental..."
8,New Export Orders,"The three industries reporting an increase in new export orders in April are: Transportation & Warehousing; Utilities; and Professional, Scientifi..."
9,Imports,"The seven industries reporting an increase in imports for the month of April, in order, are: Accommodation & Food Services; Real Estate, Rental & ..."


### Working on Data Clenaing ### 

##### 1. PMI Index Level #####

In [421]:
# variables = data_table, respondent_comment, commodity_df, ranking_df
# 1. Working on data_table

new_table = data_table.iloc[2:]
header = new_table.iloc[0]
final_table = new_table.iloc[1:]
final_table.columns = header

final_table = final_table.iloc[:,:-3]

In [422]:
new_month_level = final_table.iloc[:,1]
new_month_level = new_month_level[:-3]
new_month_level = new_month_level.to_frame().T

In [423]:
latest_date = index_data['Date'].max()
new_date = latest_date + relativedelta(months=1)
new_date_str = new_date.strftime('%Y-%m-%d')
new_month_level["Date"] = new_date
new_month_level = new_month_level[['Date'] + [col for col in new_month_level if col != 'Date']]
new_month_level.columns = index_data.columns

updated_index_data = pd.concat([index_data, new_month_level], ignore_index=True)

##### 2. What Respondents Saying #####

In [424]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""Steady improvement toward lower costs in food and beverages; however, avian bird flu may affect pricing moving forward. Already seeing increases ..."
1,"""Movie production is recovering, which should increase volume in movie theaters in the second half of 2024."" [Arts, Entertainment & Recreation]"
2,"""Although it varies by global region, we're starting to see market softening in terms of price and lead time stability. That bidders are providing..."
3,"""Inflation is raising our unit cost on products and services when compared to last year's expenditures."" [Public Administration]"
4,"""Continue to be challenged with inflationary pressure through labor and service cost increases, but we are working hard at finding utilization sav..."
5,"""Business remains soft."" [Information]"
6,"""We are still experiencing supply chain challenges with increased costs of raw materials, particularly chemicals and their containers, as well as ..."
7,"""Steady demand has been favorable during this traditionally slower season. Pricing is stable and the supply chain is strong. Employee recruitment ..."
8,"""Construction, administrative and technical/scientific labor are in high demand. Long lead times for brass plumbing fittings and electrical equipm..."
9,"""The overall market has definitely slowed down. Our business is up year over year and month over month; based on our advanced analytics, we know t..."


In [425]:
updated_comments = respondent_comment['WHAT RESPONDENTS ARE SAYING'].str.split('[', expand=True)
updated_comments.columns = [new_date_str, 'Industry']
updated_comments['Industry'] = updated_comments['Industry'].str.replace(']', '', regex=False)
updated_comments[new_date_str] = updated_comments[new_date_str].str.replace('"', '', regex=False)
updated_comments = updated_comments.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

col_order = ['Industry',new_date_str]
updated_comments = updated_comments[col_order]

final_comment = pd.merge(comments, updated_comments, on='Industry', how='left')

##### 3. Commodity Product Pricing #####

In [426]:
commodity_df

commodity_dfs = []

for index, row in commodity_df.iterrows():
    # Create a new DataFrame for the current row
    row_df = pd.DataFrame([row])
    # Append the new DataFrame to the list
    commodity_dfs.append(row_df)
    

In [427]:
commodity_up_df = commodity_dfs[0]
commodity_up_df_split = commodity_up_df['rest'].str.split(';', expand = True).T
commodity_up_header = ['Commodities Up in Price']
commodity_up_df_split.columns = commodity_up_header


commodity_down_df = commodity_dfs[1]
commodity_down_df_split = commodity_down_df['rest'].str.split(';', expand = True).T
commodity_down_header = ['Commodities Down in Price']
commodity_down_df_split.columns = commodity_down_header

commodity_short_df = commodity_dfs[2]
commodity_short_df_split = commodity_short_df['rest'].str.split(';', expand = True).T
commodity_short_header = ['Commodities in Short Supply']
commodity_short_df_split.columns = commodity_short_header

commodity_up_df

In [428]:
commodity_up_df_split[['Commodity Price Up', 'Number of Month']] = commodity_up_df_split['Commodities Up in Price'].str.split("(", n=1, expand=True)
commodity_up_df_split['Commodity Price Up'] = commodity_up_df_split['Commodity Price Up'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_up_df_split['Number of Month'] = commodity_up_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_up_df_split = commodity_up_df_split.drop(columns=['Commodities Up in Price'])

In [429]:
commodity_down_df_split

,Commodities Down in Price
0,Propane.


commodity_down_df

In [430]:
# commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
# commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
# commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
# commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])

In [431]:
if '(' in commodity_down_df_split:
    commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
    commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
    commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
    commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])
else:
    commodity_down_df_split['Number of Month'] = 1
    commodity_down_df_split.rename(columns={'Commodities Down in Price': 'Commodity Price Down'})

Commodity in short supply

In [432]:
commodity_short_df_split[['Commodities: Short Supply', 'Number of Month']] = commodity_short_df_split['Commodities in Short Supply'].str.split("(", n=1, expand=True)
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_short_df_split['Number of Month'] = commodity_short_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_short_df_split = commodity_short_df_split.drop(columns=['Commodities in Short Supply'])
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.replace('Short Supply ', '', regex=True)

Commodity Data Final Output

In [433]:
commodity_short_df_split = commodity_short_df_split.sort_values(['Number of Month'])
commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])
commodity_down_df_split = commodity_down_df_split.sort_values(['Number of Month'])

In [434]:
# commodity_short_df_split = commodity_short_df_split.sort_values(['Number of Month'])
# commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])

# if (commodity_down_df_split['Number of Month'] > 1).any():
#     commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])
# else:
#     commodity_down_df_split

##### 4. Industry Ranking #####

In [435]:
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip().str.replace('—','',regex = False).str.replace(',','',regex = False)
ranking_df_split = ranking_df['Ranking'].str.split('.', expand = True)
ranking_df_split.columns = ['Industry Up','Industry Down','No Change','None']

In [436]:
ranking_up_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Up']], axis=1)
ranking_down_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Down']], axis=1)

In [437]:
ranking_up = ranking_up_df['Industry Up'].str.split(':', expand=True)
ranking_down = ranking_down_df['Industry Down'].str.split(':', expand=True)

col_name = ['split_col','ranking_col']
ranking_up.columns = col_name
ranking_down.columns = col_name

In [438]:
filter_is_up = ranking_up['split_col'].str.split('is ', expand=True)
filter_is_down = ranking_down['split_col'].str.split('is ', expand=True)

In [439]:
filter_is_up = pd.concat([ranking_df['Industry'], filter_is_up], axis=1)
filter_is_down = pd.concat([ranking_df['Industry'], filter_is_down], axis=1)

In [440]:
def select_columns(df):
    if len(df.columns) >= 3:  # Check if DataFrame has at least 3 columns
        return df.iloc[:, [0, 2]]  # Select first and third columns
    else:
        return df.iloc[:, [0]]  # Select only the first column

filter_is_up_df = select_columns(filter_is_up)
filter_is_down_df = select_columns(filter_is_down)

In [441]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up['ranking_col']], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down['ranking_col']], axis=1)

In [442]:
ranking_up_final = pd.merge(ranking_up_final,filter_is_up_df, on= 'Industry')
ranking_down_final = pd.merge(ranking_down_final,filter_is_down_df, on= 'Industry')

In [443]:
def merge_columns(df):
    if len(df.columns) >= 3:
        df['Combined'] = df.iloc[:, 1].fillna(df.iloc[:, 2])
        df.drop(['ranking_col', 1], axis=1, inplace=True)
        return df
    else:
        df.rename(columns={'ranking_col': 'Combined'}, inplace=True)
        return df

ranking_up_final = merge_columns(ranking_up_final)
ranking_down_final = merge_columns(ranking_down_final)

In [444]:
ranking_up_final = ranking_up_final['Combined'].str.split(';', expand=True)
ranking_down_final = ranking_down_final['Combined'].str.split(';', expand=True)

In [445]:
ranking_up_final = ranking_up_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)
ranking_down_final = ranking_down_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

In [446]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up_final], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down_final], axis=1)

In [447]:
rakning_up_column_order = [ranking_up_final.columns[0]] + list(range(1, len(ranking_up_final.columns)))
rakning_down_column_order = [ranking_down_final.columns[0]] + list(range(-1, -len(ranking_down_final.columns), -1))

ranking_up_final.columns = rakning_up_column_order
ranking_down_final.columns = rakning_down_column_order

In [448]:
ranking_up_df = pd.melt(ranking_up_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')
ranking_down_df = pd.melt(ranking_down_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')

In [449]:
industry_ranking_final = pd.concat([ranking_up_df, ranking_down_df],ignore_index=True)

industry_ranking_final = industry_ranking_final.rename(columns={'Industry': 'Index','myVarname': new_date_str, 'myValname': 'Industry'})

new_col_order = ['Index','Industry',new_date_str]
industry_ranking_final = industry_ranking_final[new_col_order]


In [450]:
desired_order = ranking_df['Industry'].to_list()

industry_ranking_final['Index'] = pd.Categorical(industry_ranking_final['Index'], categories=desired_order, ordered=True)
industry_ranking_final = industry_ranking_final.sort_values(['Index',new_date_str])
latest_industry_ranking = industry_ranking_final.dropna(subset=['Industry'])

sector_ranking_final = pd.merge(sector_ranking, latest_industry_ranking, on=['Index', 'Industry'], how='left')

# Final Step - Consolidation & Exporting  #

In [451]:
final_table

2,Index,Series Index Apr,Series Index Mar,Percent Point Change,Direction,Rate of Change,Trend* (Months)
3,Services PMI®,49.4,51.4,-2.0,Contracting,From Growing,1
4,Business Activity/ Production,50.9,57.4,-6.5,Growing,Slower,47
5,New Orders,52.2,54.4,-2.2,Growing,Slower,16
6,Employment,45.9,48.5,-2.6,Contracting,Faster,3
7,Supplier Deliveries,48.5,45.4,+3.1,Faster,Slower,3
8,Inventories,53.7,45.6,+8.1,Growing,From Contracting,1
9,Prices,59.2,53.4,+5.8,Increasing,Faster,83
10,Backlog of Orders,51.1,44.8,+6.3,Growing,From Contracting,1
11,New Export Orders,47.9,52.7,-4.8,Contracting,From Growing,1
12,Imports,53.6,52.4,+1.2,Growing,Faster,4


In [452]:
commodity_up_df_split

,Commodity Price Up,Number of Month
0,Caustic Soda,1
4,Food Beverages,1
9,Labor — Technology Web Related,1
10,Office Supplies,1
12,Paper,1
13,Software Maintenance,1
2,Copper Wire,2
3,Diesel Fuel,2
14,Unleaded Fuel,2
5,Fuel,3


In [453]:
commodity_down_df_split

,Commodities Down in Price,Number of Month
0,Propane.,1


In [454]:
commodity_short_df_split

,Commodities: Short Supply,Number of Month
0,Brass Fittings,1
5,Transformers,1
2,Electrical Equipment,2
4,Switchgear,2
3,Labor,3
1,Electrical Components,4


In [455]:
updated_index_data.tail(5)

,Date,NMI,Business Activity,New Orders,Employment,Supplier Deliveries,Inventories,Prices,Backlog of Orders,Exports,Imports,Inventory Sentiment
317,2023-12-01,50.6,56.6,52.8,43.3,49.5,49.6,57.4,49.4,50.4,49.3,55.3
318,2024-01-01,53.4,55.8,55.0,50.5,52.4,49.1,64.0,51.4,56.1,59.9,59.3
319,2024-02-01,52.6,57.2,56.1,48.0,48.9,47.1,58.6,50.3,51.6,54.3,56.7
320,2024-03-01,51.4,57.4,54.4,48.5,45.4,45.6,53.4,44.8,52.7,52.4,55.7
321,2024-04-01,49.4,50.9,52.2,45.9,48.5,53.7,59.2,51.1,47.9,53.6,62.9


In [456]:
final_comment.tail(5)

,Industry,8/1/2023,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01
17,Arts Entertainment & Recreation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Finance & Insurance,NaN,NaN,NaN,NaN,NaN,Increase in activity; expecting a busy 2024.,NaN,"With the housing market continuing to stabilize, more mortgage inquiries are being made since my company opened up its mortgage loan program to lo...",NaN
19,Other Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Professional Scientific & Technical Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Real Estate Rental & Leasing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [457]:
sector_ranking_final.tail(5)

,Index,Industry,Translation,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01
193,Inventory Sentiment,Real Estate Rental & Leasing,房地产，租赁,-1.0,NaN,1.0,NaN,NaN,2.0,NaN,4.0,-1.0,NaN,1
194,Inventory Sentiment,Retail Trade,零售业,-2.0,7.0,8.0,7.0,NaN,NaN,NaN,9.0,4.0,9.0,5
195,Inventory Sentiment,Transportation & Warehousing,运输，仓储,-5.0,9.0,NaN,NaN,-1.0,-1.0,-1.0,11.0,9.0,-1.0,-1
196,Inventory Sentiment,Utilities,公共事业,4.0,10.0,4.0,4.0,3.0,7.0,8.0,6.0,7.0,2.0,7
197,Inventory Sentiment,Wholesale Trade,批发贸易,1.0,2.0,2.0,5.0,5.0,5.0,7.0,5.0,5.0,3.0,4


In [458]:
# 1. Output below does not require data overrite 

final_table.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/monthly_table.xlsx', index=False)
commodity_short_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/commmodity_short.xlsx', index=False)
commodity_up_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/commmodity_up.xlsx', index=False)
commodity_down_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/commmodity_down.xlsx', index=False)

# 2. Output below requires data overrite over the existing file 

updated_index_data.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/nmi_level.xlsx', index=False)
final_comment.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/comments.xlsx', index=False)
sector_ranking_final.to_excel('C:/Users/fanjz/Desktop/dataset/nmi/sector_ranking.xlsx', index=False)

In [ ]:
# # 1. Output below does not require data overrite 

# final_table.to_excel('/Users/jackyliu/Desktop/dataset/pmi/monthly_table.xlsx', index=False)
# commodity_short_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_short.xlsx', index=False)
# commodity_up_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_up.xlsx', index=False)
# commodity_down_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_down.xlsx', index=False)

# # 2. Output below requires data overrite over the existing file 

# updated_index_data.to_excel('/Users/jackyliu/Desktop/datase/pmi/pmi_level.xlsx', index=False)
# final_comment.to_excel('/Users/jackyliu/Desktop/dataset/pmi/comments.xlsx', index=False)
# sector_ranking_final.to_excel('/Users/jackyliu/Desktop/dataset/pmi/sector_ranking.xlsx', index=False)